# DeepStream과 MQTT를 활용한 실시간 객체 감지 데이터 전송

DeepStream을 활용하여 실시간 객체 감지 후, MQTT를 통해 감지된 데이터를 외부 시스템으로 전송하는 방법을 다룹니다. 이를 통해 IoT 장치, 클라우드 서비스, 데이터베이스 등 다양한 환경에서 실시간 감지 데이터를 활용할 수 있습니다.

## 04-2. MQTT란?
MQTT(Message Queuing Telemetry Transport)는 가벼운 메시징 프로토콜로, IoT 및 실시간 데이터 전송에 적합합니다. DeepStream에서는 감지된 객체 정보를 MQTT 브로커를 통해 퍼블리시하고, 다른 애플리케이션이 이를 구독하여 활용할 수 있습니다.

## 04-3. MQTT 활용 사례

MQTT는 경량 메시징 프로토콜로, 저지연성과 높은 신뢰성을 요구하는 다양한 산업에서 널리 사용됩니다. 다음은 대표적인 활용 사례입니다.

### 1. IoT 및 스마트 홈
* 스마트 조명, 온도 센서, 도어락과 같은 IoT 디바이스에서 상태 데이터를 전송하고 원격으로 제어하는 데 사용됩니다.
* 예: 스마트홈 시스템에서 MQTT를 활용하여 사용자의 스마트폰과 가전기기 간 통신을 수행.

### 2. 산업 자동화
* 제조업에서 센서 및 기계 데이터를 실시간으로 모니터링하고 원격으로 제어하는 데 활용됩니다.
* 예: 공장의 IoT 센서가 온도, 습도, 기계 작동 상태를 MQTT를 통해 중앙 제어 서버에 전송.

### 3. 차량 및 물류 시스템
* 차량의 위치, 속도, 연료 상태 등의 데이터를 MQTT를 통해 클라우드 서버로 전송하여 실시간 관리를 가능하게 합니다.
* 예: 물류 트래킹 시스템에서 차량이 특정 위치에 도착했을 때 MQTT 메시지를 발행하여 자동으로 배송 상태 업데이트.

### 4. 헬스케어 및 원격 진료
* 병원 내 환자의 바이탈 데이터(심박수, 혈압, 체온 등)를 실시간으로 의료진에게 전송하여 모니터링할 수 있습니다.
* 예: 스마트 의료기기가 환자의 생체 정보를 MQTT를 이용해 클라우드로 전송하고 의료진이 원격으로 모니터링.

### 5. 스마트 시티 및 공공 인프라
* 교통량 감지, 대기 오염 모니터링, 스마트 가로등 제어 등 도시 관리 시스템에서 MQTT가 활용됩니다.
* 예: 교통 신호 체계가 MQTT를 통해 차량 흐름 데이터를 수집하여 실시간 교통 패턴 최적화.

### 6. AI 및 Edge Computing
* AI 모델의 추론 결과를 실시간으로 다른 시스템에 공유하기 위해 MQTT를 사용할 수 있습니다.
* 예: DeepStream이 객체 감지 결과를 MQTT를 통해 클라우드 서버나 데이터베이스로 전송하여 실시간 분석.

---

# 실습
## MQTT 환경 설정
### 1. Mosquitto MQTT 브로커 설치
```bash
$ sudo apt update
$ sudo apt install mosquitto mosquitto-clients
```

Mosquitto 서비스 활성화:
```bash
$ sudo systemctl start mosquitto
$ sudo systemctl enable mosquitto
```

### 2. MQTT Python 클라이언트(paho-mqtt) 설치
```bash
$ pip install paho-mqtt
```

## DeepStream과 MQTT 연동
### 1. DeepStream 메시지 브로커 설정
DeepStream에서는 `nvds_msgbroker` 플러그인을 사용하여 MQTT로 데이터를 전송할 수 있습니다.

**설정 파일 예시 (`config_mqtt.txt`)**:
```ini
[message-broker]
proto-lib=/opt/nvidia/deepstream/deepstream/lib/libnvds_mqtt_proto.so
conn-str=localhost;1883;deepstream_topic
config-file=config_mqtt.json
```

**MQTT 설정 파일 예시 (`config_mqtt.json`)**:
```json
{
    "mqtt_broker": "localhost",
    "mqtt_port": 1883,
    "topic": "deepstream_topic",
    "client_id": "deepstream_client",
    "qos": 1
}
```

### 2. DeepStream 실행
```bash
$ deepstream-app -c config_mqtt.txt
```

이제 감지된 객체 정보가 MQTT 브로커를 통해 `deepstream_topic`으로 전송됩니다.

## MQTT 데이터 수신
### 1. Python MQTT Subscriber 작성
MQTT 데이터를 구독하고 출력하는 간단한 스크립트:
```python
import paho.mqtt.client as mqtt

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("deepstream_topic")

def on_message(client, userdata, msg):
    print(f"Received message: {msg.payload.decode()}")

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("localhost", 1883, 60)
client.loop_forever()
```